In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
wikipage_can = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
wikipage_can_data = BeautifulSoup(wikipage_can,'lxml') # pull data out of wikipage using bs4
table_can_postalcode = wikipage_can_data.find('table') #find the portion with table with postal code and other details
column_values = table_can_postalcode.find_all('td') # capture all column values with <td> tag

element_count = len(column_values)

postcode = []
borough = []
neighborhood = []

for i in range(0, element_count, 3):
    postcode.append(column_values[i].text.strip())
    borough.append(column_values[i+1].text.strip())
    neighborhood.append(column_values[i+2].text.strip())

In [3]:
df_can_postalcode = pd.DataFrame(data=[postcode, borough, neighborhood]).transpose()
df_can_postalcode.columns = ['Postcode', 'Borough', 'Neighborhood']

In [4]:
#Remove elements with Not assigned neighborhood.
df_can_postalcode.drop(df_can_postalcode[df_can_postalcode['Borough'] == 'Not assigned'].index, inplace=True)
df_can_postalcode.loc[df_can_postalcode.Neighborhood == 'Not assigned', "Neighborhood"] = df_can_postalcode.Borough

In [5]:
# Group the neighbourhood with same postal code
df_can = df_can_postalcode.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_can.columns = ['Postcode', 'Borough', 'Neighborhood']

In [6]:
df_can.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
df_can.shape

(103, 3)

In [9]:
df_lat_long = pd.read_csv('https://cocl.us/Geospatial_data')
df_lat_long.columns = list(map(str, df_lat_long.columns))
df_lat_long.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df_can_postcode_Lat_long=pd.merge(df_can, df_lat_long, on='Postcode', how='outer')
#df_can_postcode_Lat_long

In [10]:
toronto_data = df_can_postcode_Lat_long[df_can_postcode_Lat_long['Borough'].str.contains('Toronto')].reset_index(drop=True)
#toronto_data

In [11]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [12]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [13]:
# create map of New York using latitude and longitude values
map_can = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_can)  
    
map_can

In [14]:
#Defining Foursquare credential and 
CLIENT_ID = 'HUZKQA4G4BDMP11PECKG4SS00TZLK1AXU2U3T1JBAEUQTULE' # your Foursquare ID
CLIENT_SECRET = '0O44S52G3KF0JRJUQ2YFV51P15JKNZDQJQ0HAJ30OZQFM1N5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HUZKQA4G4BDMP11PECKG4SS00TZLK1AXU2U3T1JBAEUQTULE
CLIENT_SECRET:0O44S52G3KF0JRJUQ2YFV51P15JKNZDQJQ0HAJ30OZQFM1N5


In [44]:
toronto_data.loc[6, 'Neighborhood']

'North Toronto West'

In [38]:
#Now, let's get the top 100 venues that are in "North Toronto West" within a radius of 500 meters.
neighborhood_latitude = toronto_data.loc[6, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[6, 'Longitude'] # neighborhood longitude value

In [39]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=HUZKQA4G4BDMP11PECKG4SS00TZLK1AXU2U3T1JBAEUQTULE&client_secret=0O44S52G3KF0JRJUQ2YFV51P15JKNZDQJQ0HAJ30OZQFM1N5&v=20180605&ll=43.7153834,-79.40567840000001&radius=500&limit=100'

In [40]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ca39df69fb6b714157ec249'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Lawrence Park South',
  'headerFullLocation': 'Lawrence Park South, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 20,
  'suggestedBounds': {'ne': {'lat': 43.7198834045, 'lng': -79.39946407145794},
   'sw': {'lat': 43.7108833955, 'lng': -79.41189272854209}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '51606062e4b0878cf540f4a2',
       'name': 'Barreworks',
       'location': {'address': '2576 Yonge St',
        'lat': 43.71407030751952,
        'lng': -79.40010911522093,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.7

In [41]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [42]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Barreworks,Yoga Studio,43.714070,-79.400109
1,Uncle Betty's Diner,Diner,43.714452,-79.400091
2,Civello Salon,Salon / Barbershop,43.715111,-79.400304
3,lululemon,Sporting Goods Shop,43.715720,-79.399960
4,Starbucks,Coffee Shop,43.715456,-79.400303


In [43]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

20 venues were returned by Foursquare.


In [45]:
#Explore Neighborhoods in North Toronto West. 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [46]:
#Call function and display the venue list around North Toronto West. 
toronto_venue = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 